In [1]:
import pandas as pd
import pickle

In [2]:
st_dados = pd.read_pickle("../parsed-data/dados.pd.pkl")

In [3]:
st_metadados = pd.read_pickle("../parsed-data/metadados.pd.pkl")  

In [4]:
geocodes = pd.read_pickle("../parsed-data/geocodes.pd.pkl")

In [5]:
ceps = pd.read_csv("../data/geoloc/CEPdatabase.csv")
ceps.set_index("cep", inplace=True)

In [6]:
regioes = pd.read_csv("../data/geoloc/regioes-saude.csv")
regioes.set_index("CODIBGE", inplace=True)

In [7]:
tipo_unidade = pd.read_csv("../data/estabelecimentos/tipo_unidade.csv")
tipo_unidade.set_index("cod", inplace=True)

In [8]:
CEPS_ERRADOS = []
def pass_cep(codigo):
    try:
        x,y = ceps.loc[codigo,"latitude":"longitude"]
    except KeyError: 
        print(f"Erro com {codigo}")
        CEPS_ERRADOS.append(codigo)
        x,y = None, None
    return [x,y]

In [9]:
st_dados.set_index("CNES",inplace=True)

In [10]:
ST = pd.read_csv("../data/estabelecimentos/estabelecimentos-420000-202202.csv")

In [11]:
ST["TP_UNID"] = ST.CNES.apply(lambda x: st_dados.loc[x, "TP_UNID"])
ST["tipo_unidade"] = ST.TP_UNID.apply(lambda x: tipo_unidade.loc[x,"tipo_unidade"])
latlong = ST.apply( lambda x: pass_cep(x["CEP"]), axis=1, result_type="expand" )
ST["lat"] = latlong.loc[:,0]
ST["lon"] = latlong.loc[:,1]
ST["regiao"] = ST.IBGE.apply(lambda x: regioes.loc[x,"REGIAO"])

Erro com 89460000
Erro com 89560000
Erro com 89560000
Erro com 89560000
Erro com 89010580
Erro com 89812758
Erro com 88113330
Erro com 89012201
Erro com 88117907
Erro com 89812211
Erro com 89560000
Erro com 88113330
Erro com 88117902
Erro com 88807213
Erro com 89500993
Erro com 89560000
Erro com 89560000
Erro com 88113330
Erro com 88117907
Erro com 89560000
Erro com 89052902
Erro com 89300000


In [12]:
ST.update(geocodes)

In [18]:
ST[~ST.lat.isna()].to_pickle("../parsed-data/estabelecimentos.pd.pkl")

In [42]:
ST.loc[ST.lat.isna(),["lat","lon"]] = ST[ST.lat.isna()].apply(lambda x: ST.query(f"IBGE == {x['IBGE']}")[["lat","lon"]].sample(n=1).values[0], axis=1, result_type="expand")

In [50]:
def get_lat_lon(idx):
    ibge = ST.loc[idx,"IBGE"]
    sample = ST.query(f"IBGE == {ibge}").sample(n=1).copy()
    return sample[["lat","lon"]].values[0]

for x in ST.loc[ST.lat.isna()].index:
    lat, lon = get_lat_lon(x)
    ST.loc[x,"lat"] = lat
    ST.loc[x,"lon"] = lon